<a href="https://colab.research.google.com/github/Karim-Anwar/MasterThesis/blob/main/Master_Conv_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from __future__ import print_function, division

import argparse
import datetime
import os
import os.path as osp
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from torch import save, load
from torch.nn import Conv2d
from torch.nn import Linear


In [3]:
#channels (1 to 3) size = sample size, height, width
torch.randint(1, 4, size=(5,4,5))

tensor([[[2, 2, 3, 3, 1],
         [1, 3, 1, 3, 1],
         [1, 1, 3, 3, 3],
         [2, 1, 3, 1, 3]],

        [[1, 1, 2, 2, 1],
         [1, 2, 2, 1, 2],
         [1, 2, 1, 1, 1],
         [3, 3, 1, 1, 1]],

        [[3, 1, 2, 3, 1],
         [3, 2, 3, 3, 3],
         [3, 2, 1, 1, 2],
         [2, 3, 3, 1, 1]],

        [[2, 2, 3, 3, 1],
         [2, 1, 2, 2, 2],
         [3, 1, 2, 1, 1],
         [2, 2, 1, 1, 3]],

        [[2, 1, 2, 3, 1],
         [3, 3, 1, 3, 1],
         [2, 2, 3, 2, 3],
         [2, 2, 2, 2, 2]]])

In [4]:
# Set random seed for reproducibility
torch.manual_seed(42)

width = 5
height = 4
channels = 3
sample_size = 100
rand_images = torch.rand(sample_size, width, height, channels)
rand_labels = torch.randint(0,3, size=(sample_size,))

In [5]:
rand_images

tensor([[[[0.8823, 0.9150, 0.3829],
          [0.9593, 0.3904, 0.6009],
          [0.2566, 0.7936, 0.9408],
          [0.1332, 0.9346, 0.5936]],

         [[0.8694, 0.5677, 0.7411],
          [0.4294, 0.8854, 0.5739],
          [0.2666, 0.6274, 0.2696],
          [0.4414, 0.2969, 0.8317]],

         [[0.1053, 0.2695, 0.3588],
          [0.1994, 0.5472, 0.0062],
          [0.9516, 0.0753, 0.8860],
          [0.5832, 0.3376, 0.8090]],

         [[0.5779, 0.9040, 0.5547],
          [0.3423, 0.6343, 0.3644],
          [0.7104, 0.9464, 0.7890],
          [0.2814, 0.7886, 0.5895]],

         [[0.7539, 0.1952, 0.0050],
          [0.3068, 0.1165, 0.9103],
          [0.6440, 0.7071, 0.6581],
          [0.4913, 0.8913, 0.1447]]],


        [[[0.5315, 0.1587, 0.6542],
          [0.3278, 0.6532, 0.3958],
          [0.9147, 0.2036, 0.2018],
          [0.2018, 0.9497, 0.6666]],

         [[0.9811, 0.0874, 0.0041],
          [0.1088, 0.1637, 0.7025],
          [0.6790, 0.9155, 0.2418],
          [0.1

In [6]:
rand_labels

tensor([2, 1, 0, 0, 0, 2, 0, 2, 2, 1, 0, 1, 1, 2, 0, 1, 1, 1, 2, 2, 1, 0, 1, 0,
        2, 0, 2, 1, 0, 0, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 1, 2, 0, 0, 1, 2, 0,
        0, 1, 1, 0, 0, 2, 1, 0, 2, 2, 0, 0, 0, 2, 0, 2, 1, 0, 1, 1, 2, 1, 1, 2,
        2, 2, 0, 1, 2, 1, 0, 2, 1, 0, 1, 0, 2, 2, 2, 1, 0, 0, 0, 2, 2, 0, 0, 1,
        0, 1, 0, 2])

In [7]:
class RandDataSet(Dataset):
  def __init__(self, images, labels):
    self.images = images
    self.labels = labels

  def __len__(self):
    return len(self.images)

  def __getitem__(self, idx):
    image = self.images[idx]
    label = self.labels[idx]
    return image, label



In [8]:
dataset = RandDataSet(rand_images, rand_labels)
dataloader = DataLoader(dataset, batch_size=20, shuffle = True)

In [9]:
# training_data = torch.utils.data.TensorData(input_data, output_lables)

In [14]:
class Convolutional2Layer(nn.Module):
  def __init__(self, channels, height, width):
    super(Convolutional2Layer, self).__init__()
    # following the basic structure of a CNN we add a secondary conv layer after pooling to get more detailed features
    # intializing the layers requires knowledge on the dataset and multiple testing (ask about it in meeting)
    self.conv1 = nn.Conv2d(in_channels=channels, out_channels=16, kernel_size=3, padding=1)
    self.pool1 = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
    # do you maxpool again after convolving again (or do we loose to much information)
    self.pool2 = nn.MaxPool2d(2,2)
    # we divide by foor the dimensions since we max pool twice, 3 is the output classes
    self.fc = nn.Linear(32 * width // 4 * height // 4, 3)

  def forward(self):
    x = self.pool1(F.relu(self.conv1(x)))
    x = self.pool2(F.relu(self.conv2(x)))\
    # view(-1) flattens the tensor without modifing it
    x = x.view(-1, 32 * (height // 4) * (width // 4))
    x = self.fc(x)
    return x


In [16]:
cnn = Convolutional2Layer(channels=channels, height=height, width=width)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.01, momentum=0.9)